In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow import data as tfdata
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow import losses
from tensorflow.keras import initializers as init

from tensorflow.keras.utils import plot_model

# 变量的保存与恢复
### tf.train.Checkpoint
用处： 
    很多时候我们希望将模型训练完成后将训练好的参数变量保存骑来，在需要使用模型的其他地方载入模型和参数，就能直接得到训练好的参数   
现状：
    tensorflow的变量类型 ResourceVariable 不能被序列化  
解决：
    tensorflow提供了tf.train.Checkpoint 这个类来进行变量保存和恢复  
    可以使用save() 和 restore() 方法将tensorflow中所有包含Checkpointable State的对象进行保存和恢复，  
    - tf.keras.optimizer  
    - tf.ResourceVariable  
    - tf.keras.Layer  
    - tf.keras.Model   
    均可以被保存


使用方法
1、首先声明一个checkpoint对象
```python
    checkpoint = tf.train.Checkpoint(model=model)
```
Checkpoint接受的初始化参数比较特殊，是一个**kwargs, 即一系列的kv对， 键名可以随便取，value为需要保存的对象  
例如，保存一个继承了tf.keras.Model的模型实例，和一个继承了tf.train.Optimizer的优化器  
```python
    checkpoint = tf.train.Checkpoint(myModel=model, myOptimizer=myOptimizer)
```

当模型训练完成需要保存的时候，使用  
```python
    checkpoint.save(save_path_with_prefix)
```

这里的save_path_with_prefix是保存文件的目录+前缀  

checkpoint.save() 可以进行多次调用，每次调用救护得到一个.index和.data文件，序号依次增加


<big>checkpoint只能保存模型的参数，不能保存模型的计算过程, 一般用于在具有模型源代码的时候恢复之前训练好的模型参数</big>

当在其他地方需要重新载入一个之前保存的参数时，需要再次实例化一个checkpoint，同时保持键名一致， 再次调用restore方法。
```python
    model_to_be_restored = MyModel() #待恢复参数的同一模型
    checkpoint = tf.train.Checkpoint(Mymodel=model_to_be_restored) # 保持键名一致
    checkpoint.restore(save_path_with_prefix_and_index)
```

tf.train.Checkpoint相比于 之前版本的 tf.train.Saver相比 
支持即时执行模式下 『延迟』恢复变量，  
即： 当调用了checkpoint.restore()， 但模型中的变量还没有被建立的时候，Checkpoint可以 等到变量被建立的时候再进行数值恢复  
即时执行模式下，模型中各个层的初始化和变量是在模型第一次被调用的时候才进行的

## 使用tf,train.CheckpointManager

使用场景：  
   &nbsp;&nbsp; - 多次训练后累积大量的checkpoint，但是只需保留最后几个  
   &nbsp;&nbsp; - 默认编号从1开始，希望使用别的编号方式   
使用方法：  
   &nbsp;&nbsp; 在定义的Checkpoint后接着定义一个CheckpointManager  

In [ ]:
checkpoint = tf.train.Checkpoint(model=model)
checkpoint_manager = tf.train.CheckpointManager(checkpoint, directory='./save', checkpoint_name="model.ckpt", max_to_keep=k)

其中  
&nbsp;&nbsp; - directory: 文件保存路径  
&nbsp;&nbsp; - checkpoint_name： 文件前缀名  
&nbsp;&nbsp; - max_to_keep : 保留的checkpoint数目  

此外  
&nbsp;&nbsp;manager.save()可以接受一个checkpoint_number=xxx 来指定编号的命名

# 训练过程可视化 ： TensorBoard


- 1、首先在代码目录下建立一个文件夹, 如./log   
- 2、实例化一个记录器： ```ummart_writer = tf.summary.create_file_writer("./log")```
- 3、通过with语句指定希望使用的记录器
- 4、对需要记录的参数执行 ```tf.summary,scalar(name, tensor, step=batch_index)```
    step参数可以根据需要自行设置
- 5、在代码目录终端执行 tensorboard --logdir=./log
- 6、在浏览器访问命令行输出的地址

每运行依次tf.summary.scalar()， 记录器就会相记录文件中写入一条记录。

### 查看Graph和Profile的信息
还可以在训练时使用 tf.summary.trace_on 来开启Trace。  
此时tensorflow会将训练的大量信息（如计算的图结构，每个操作的耗时）等记录下   
训练完成后使用tf.summary.trace_export将记录结果输出到文件

In [ ]:
# 使用如下
tf.summary.trace_on(graph=True, profiler=True)
# 进行训练
with tf.summary.create_file_writer("./log").as_default():
    # 保存trace信息到文件
    tf.summary.trace_export(name="model_trace", step=0, profiler_outdir=log_dir)

# 数据集的构建和预处理 tf.data

tf.data包含了一套灵活的数据集构建api,能够帮助我们快速、高效的构建数据输入的流水线，尤其适用于数据量巨大的场景